In [ ]:
from probability_distributions import *
import matplotlib.pyplot as plt

from probability_distributions import * 
from utils import *
from model import *
from train import *
from dataset import *

In [ ]:
img_size = (64, 64)
p_radius = P_Radius(high=img_size[0]//2)
radius = p_radius.sample()

p_brightness = P_Brightness(high=255)
brightness = p_brightness.sample()

p_x_under_f = P_X_under_F(radius, brightness, std=25, size=img_size)
x = p_x_under_f.sample()

plt.imshow(x, cmap="gray")
plt.show()

In [ ]:
P_F_under_X(p_x_under_f, p_radius, p_brightness).loglikelihood(x, radius, brightness)

In [ ]:
### Create training data
n = 10000
noise_std = 25
radius = []
brightness = []
x = []
for _ in range(n):
    radius.append(p_radius.sample())
    brightness.append(p_brightness.sample())
    p_x_under_f = P_X_under_F(radius[-1], brightness[-1], std=noise_std, size=img_size)
    x.append(p_x_under_f.sample())

In [ ]:
plt.imshow(x[5], cmap="gray")

In [ ]:
dataset = Dataset(x, radius, brightness)
train_set, val_set = torch.utils.data.random_split(dataset, [0.8, 0.2])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_mean = torch.stack(x).mean()
x_std = torch.stack(x).std()

# Create model
model = CNN(8, x_mean, x_std, img_size=img_size, n_gaussians=1, n_features=3)
# Create optimiser
optimizer = torch.optim.Adamax(model.parameters())
# Train
train_losses, val_losses = train(
    model,
    optimizer,
    train_loader,
    val_loader,
    n_epochs=200,
    device=device,
)

In [ ]:
plt.plot(train_losses[1:], label="train")
plt.plot(val_losses[1:], label="val")
plt.legend()
plt.show()